#### Задание 1
Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса https://www.cbr-xml-daily.ru/daily_json.js

In [27]:
import requests
import json

In [28]:
def max_currency():
    #словарь название - курс
    full_data = requests.get('https://www.cbr-xml-daily.ru/daily_json.js').json()
    currency_dict = {}
    for currency in full_data['Valute'].keys():
        currency_dict[full_data['Valute'][currency]['Name']] = full_data['Valute'][currency]['Value']
    
    #название валюты с макс.значением курса
    for currency in currency_dict:
        if currency_dict.get(currency) == max(currency_dict.values()):
            print(currency)    

In [29]:
max_currency()

СДР (специальные права заимствования)


#### Задание 2
Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах курсов валют (eur, usd итд) будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [31]:
class Rate:
    def __init__(self, format_='value', diff_=False):
        self.format = format_
        self.diff = diff_
        
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format == 'full':
                return response[currency]
            
            if self.format == 'value' and self.diff == False:
                return response[currency]['Value']

            if self.format == 'value' and self.diff == True:
                return response[currency]['Previous']            
            
        return 'Error'
    
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')
    
    def brl(self):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')

In [32]:
r = Rate()
r.eur()

89.2576

In [35]:
r1 = Rate(format_='full')
r1.eur()

{'ID': 'R01239',
 'NumCode': '978',
 'CharCode': 'EUR',
 'Nominal': 1,
 'Name': 'Евро',
 'Value': 89.2576,
 'Previous': 89.9768}

In [40]:
r2 = Rate(diff_=True)
r2.eur()

89.9768

#### Задание 3
Напишите класс Designer, который учитывает количество международных премий. Подсказки в коде занятия в разделе “Домашнее задание задача 3”.

Комментарий по классу Designer такой:
Напишите класс Designer, который учитывает количество международных премий для дизайнеров (из презентации: “Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла”). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять).

Класс Designer пишется по аналогии с классом Developer из материалов занятия. Комментарий про его условия Вика написала выше: “Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла”

In [68]:
# класс из лекции
class Employee:
    def __init__(self, name, seniority):
        self.name = name
        self.seniority = seniority

        self.grade = 1
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)
    
    def check_if_it_is_time_for_upgrade(self):
        pass

In [73]:
# класс Разработчки из лекции
class Developer(Employee):
    def __init__(self, name, seniority):
        super().__init__(name, seniority)
    
    def check_if_it_is_time_for_upgrade(self):
        # для каждой аккредитации увеличиваем счетчик на 1
        # пока считаем, что все разработчики проходят аккредитацию
        self.seniority += 1
        
        # условие повышения сотрудника из презентации
        if self.seniority % 5 == 0:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()

In [74]:
# проверяем как работает система повышения сотрудников на примере отдела разработки

# разработчик Александр только что пришел в компанию
alex = Developer('Александр', 0)

In [75]:
for i in range(15):
    alex.check_if_it_is_time_for_upgrade()

Александр 1
Александр 1
Александр 1
Александр 1
Александр 2
Александр 2
Александр 2
Александр 2
Александр 2
Александр 3
Александр 3
Александр 3
Александр 3
Александр 3
Александр 4


In [76]:
print(alex.name)
print(alex.seniority)

Александр
15


In [97]:
# класс Дизайнеры - ДЗ
class Designer(Employee):
    def __init__(self, name, seniority, prize):
        super().__init__(name, seniority)
        self.prize = prize

    def prize_up(self):
        """Повышает кол-во призов у сотрудника"""
        #дополнительно, но условиям задачи не используем, кол-во призов не меняется / можно будет 
        self.prize += 1        
        
    def check_if_it_is_time_for_upgrade(self):
        # для каждой аккредитации увеличиваем счетчик на 1
        # считаем, что дизайнеры проходят аккредитацию как и разработчики
        self.seniority += 1
        
        # условие повышения сотрудника из презентации
        if (self.seniority + self.prize * 2) % 7 == 0:
            self.grade_up()
        
        # публикация результатов
        return self.publish_grade()

In [98]:
olga = Designer('Ольга',0,2)
print(olga.name)
print(olga.seniority)
print(olga.prize)

Ольга
0
2


In [99]:
for i in range(15):
    olga.check_if_it_is_time_for_upgrade()

Ольга 1
Ольга 1
Ольга 2
Ольга 2
Ольга 2
Ольга 2
Ольга 2
Ольга 2
Ольга 2
Ольга 3
Ольга 3
Ольга 3
Ольга 3
Ольга 3
Ольга 3
